<a href="https://colab.research.google.com/github/Amirosein/MilkywayFormation/blob/main/MilkyWay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io, requests, zipfile

!pip install tsfresh
from tsfresh import extract_features

!pip install pyunpack
!pip install patool
from pyunpack import Archive

#import warnings
#warnings.filterwarnings('ignore')


# plt.rcParams.update({'font.size': 20, 'figsize':(8,6)})
SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rc('axes', linewidth = 5 )
plt.rc('lines', linewidth = 5 )

plt.rc('lines', markersize = 5 )

plt.rc('figure', figsize=(8,4) )         # Image size

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 81kB 3.6MB/s 


# Mount drive

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# **Convert Dat. files to a CSV Function**


In [3]:
import pandas as pd
from pathlib import Path
import csv
from pathlib import Path

f = open("/content/draft.csv", "x")

# function
def data2csv(path, var, os):
  files = Path(path).rglob("*.dat")

  i = 0

  for file in files:
    datContent = [i.strip().split() for i in open(file).readlines()]
    id = os +'-'+ Path(file).stem
    
    # write it as a new CSV file
    with open("/content/draft.csv", "w") as f:
      writer = csv.writer(f)
      writer.writerows(datContent)

    df = pd.read_csv("/content/draft.csv", names=['time','value','e']) 
    df = df[['time','value']]
    df['id'] = id
    df['kind'] = var

    if i==0:
      data = df
    else:
      data = pd.concat([data,df], axis=0)

    i = 1

  return data

> * **Save file**

In [4]:
def savefile(df,name):
  t = df
  t.to_csv(name, header=True, index=True)
  return 0

# Convert and Save data

In [5]:
Archive('/content/drive/MyDrive/IIILMC/phot.tar.gz').extractall('/content/drive/MyDrive/IIILMC')
#Archive('/content/drive/MyDrive/IVLMC/phot.tar.gz').extractall('/content/drive/MyDrive/IVLMC')
#Archive('/content/drive/MyDrive/III/phot.tar.gz').extractall('/content/drive/MyDrive/III')
#Archive('/content/drive/MyDrive/IV/phot.tar.gz').extractall('/content/drive/MyDrive/IV')

In [6]:
LMC3_I = data2csv('/content/drive/MyDrive/IIILMC/phot/I', 'I','III')
LMC3_V = data2csv('/content/drive/MyDrive/IIILMC/phot/V', 'V','III')

#LMC4_I = data2csv('/content/drive/MyDrive/IVLMC/phot/I', 'I','IV')
#LMC4_V = data2csv('/content/drive/MyDrive/IVLMC/phot/V', 'V','IV')

#SMC3_I = data2csv('/content/drive/MyDrive/III/phot/I', 'I','III')
#SMC3_V = data2csv('/content/drive/MyDrive/III/phot/V', 'V','III')

#SMC4_I = data2csv('/content/drive/MyDrive/IV/phot/I', 'I','IV')
#SMC4_V = data2csv('/content/drive/MyDrive/IV/phot/V', 'V','IV')

In [7]:
LMC3 = pd.concat([LMC3_I,LMC3_V])
#LMC4 = pd.concat([LMC4_I,LMC4_V])
#SMC3 = pd.concat([SMC3_I,SMC3_V])
#SMC4 = pd.concat([SMC4_I,SMC4_V])

DPV = pd.concat([LMC3])

DPV['class'] = 'DPV'
savefile(DPV,'DPV')
DPV.head()

,time,value,id,kind,class
0,2184.81184,17.946,III-OGLE-LMC-CEP-0001,I,CEP
1,2188.86239,17.631,III-OGLE-LMC-CEP-0001,I,CEP
2,2191.77255,17.858,III-OGLE-LMC-CEP-0001,I,CEP
3,2200.79366,17.759,III-OGLE-LMC-CEP-0001,I,CEP
4,2206.72879,17.686,III-OGLE-LMC-CEP-0001,I,CEP


# Feature Extraction Function

In [8]:
def feat(X):
  m = extract_features(X, column_id="id", column_sort="time",
                               column_kind="kind", column_value="value")
#  m['class'] = cls
  return m

In [ ]:
acep = feat(CEP)
acep.head()

# Outputs Encoding

> * **One-Hot Encoding**

In [ ]:
from sklearn.preprocessing import OneHotEncoder

a = np.array(data['class'])
OH_Enc = OneHotEncoder( )
OH_Encoded = OH_Enc.fit_transform(a.reshape(-1,1)).toarray()
OH_Outputs = pd.DataFrame(OH_Encoded)
OH_Outputs.columns = ['OH1','OH2','OH3','OH4','OH5']

data = pd.concat([data,OH_Outputs], axis=1)
data.head()

#a = OH_Enc.inverse_transform(np.array([[1, 0, 0, 0, 0],[0, 1, 0, 0, 0],[0, 0, 1, 0, 0]
             #             ,[0, 0, 0, 1, 0],[0, 0, 0, 0, 1]]))

#print('[1, 0, 0, 0, 0] =',a[0],'\n')
#print('[0, 1, 0, 0, 0] =',a[1],'\n')
#print('[0, 0, 1, 0, 0] =',a[2],'\n')
#print('[0, 0, 0, 1, 0] =',a[3],'\n')
#print('[0, 0, 0, 0, 1] =',a[4],'\n')

> * **Ordinal Encoding**

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(data['class'])
Ordinal_Y = le.transform(data['class']) 
Y_O = pd.DataFrame(Ordinal_Y)
Y_O.columns = ['Y']

data = pd.concat([data,Y_O], axis=1)
data.head()

#le.inverse_transform([0, 1, 2, 3, 4])

# Shuffle and Imputation

> * Shuffle

In [ ]:
from sklearn.utils import shuffle

data = shuffle(data, random_state=42)
data = data.reset_index()

X = data.loc[:,'Jmag (1.22u)':'160u flux']
Y = data.loc[:,'y1':'y5']
categories = data['class']

data.head()

> * Iterative Imputer

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imp = IterativeImputer()
imp_data = imp.fit_transform(X)
X = pd.DataFrame(imp_data, columns= X.columns)